In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_base.utils import formatted_age, get_utcnow
from dateutil.relativedelta import relativedelta

In [2]:
#Helper Methods
from edc_base.utils import formatted_age, get_utcnow, age

today = get_utcnow()- relativedelta(days=3)

def age_at_consent(child_consent):
    child_age = age(child_consent.child_dob, child_consent.consent_datetime)
    child_age = str(child_age.years) + '.' + str(child_age.months)
    return child_age

def get_current_age(child_consent):
    child_age = age(child_consent.child_dob, today)
    child_age = str(child_age.years) + '.' + str(child_age.months)
    return child_age


def get_child_dataset(child_consent):
    try:
        child_dataset = ChildDataset.objects.get(study_child_identifier=child_consent.study_child_identifier)
    except ChildDataset.DoesNotExist:
        print('welp!')
    else:
        return child_dataset
    
def get_prev_study(child_consent):
    
    child_dataset = get_child_dataset(child_consent)
    if child_dataset:
        try:
            md = MaternalDataset.objects.get(study_maternal_identifier=child_dataset.study_maternal_identifier)
        except MaternalDataset.DoesNotExist:
            print('yeeiii!!')
        else:
            return md.protocol
    

def get_exposure_status(child_consent):
    child_dataset = get_child_dataset(child_consent)
    if child_dataset:
        if child_dataset.infant_hiv_exposed.lower() == 'exposed':
            return 'HEU'
        elif child_dataset.infant_hiv_exposed.lower() == 'unexposed':
            return 'HUU' 

def get_arvs_exposed(child_consent):
    
    try:
        madp = MaternalArvDuringPreg.objects.get(
            maternal_visit__subject_identifier=child_consent.subject_identifier[:-3])
    except MaternalArvDuringPreg.DoesNotExist:
        pass
    else:
        return list(set(madp.maternalarvtableduringpreg_set.all().values_list('arv_code')))
    
def get_preg_arv_exposed(child_consent):
    child_dataset = get_child_dataset(child_consent)
    if child_dataset:
        try:
            md = MaternalDataset.objects.get(study_maternal_identifier=child_dataset.study_maternal_identifier)
        except MaternalDataset.DoesNotExist:
            print('yeeiii!!')
        else:
            arv_list = []
            if md.preg_dtg:
                arv_list.append('preg_dtg')
            if md.preg_pi:
                arv_list.append('preg_pi')
            if md.preg_efv:
                arv_list.append('preg_efv')
            return arv_list
        
def get_part_info(cohort_child_pids):
    
    data=[['child_pid', 'caregiver_pid', 'DOB', 'consent_date', 'age_at_consent', 'age_today', 'cohort',
      'exposure_status', 'prev_study', 'arvs_exposed']]

    for a in cohort_child_pids:
        try:
            child_consent = CaregiverChildConsent.objects.filter(subject_identifier=a).latest('consent_datetime')
        except CaregiverChildConsent.DoesNotExist:
            print('hmmmmm?')
        else:
            preg_arvs = get_arvs_exposed(child_consent) or []

            preg_arvs = preg_arvs + get_preg_arv_exposed(child_consent)

            x = [child_consent.subject_identifier, child_consent.subject_identifier[:-3],
                child_consent.child_dob, child_consent.consent_datetime.date(), age_at_consent(child_consent),
                get_current_age(child_consent), child_consent.cohort,
                get_exposure_status(child_consent),  get_prev_study(child_consent), preg_arvs]
            data.append(x)
    return data



In [3]:
#cohort_a
childa_dob_limit = today - relativedelta(years=5)

cca = CaregiverChildConsent.objects.filter(cohort__startswith='cohort_a',
                                           child_dob__lte=childa_dob_limit.date()
                                          ).values_list('subject_identifier', flat=True).distinct()

cca.count()


32

In [5]:
#get cohort A data
adata = get_part_info(cca)

import csv
path = '/home/flourish/cohort_a_age_out.csv'
# path = '/Users/imosweu/cohort_a_age_out.csv'

with open(path, 'w') as result_file:
    wr = csv.writer(result_file, dialect='excel')

    for q in adata:
         wr.writerow(q)

In [8]:
#cohort_b
childb_dob_limit = today - relativedelta(years=10, months=5)

ccb = CaregiverChildConsent.objects.filter(cohort__startswith='cohort_b',
                                           child_dob__lte=childb_dob_limit.date()
                                          ).values_list('subject_identifier', flat=True).distinct()

ccb.count()


102

In [9]:
#get cohort B data
bdata = get_part_info(ccb)

import csv
path = '/home/flourish/cohort_b_age_out.csv'
# path = '/Users/imosweu/cohort_b_age_out.csv'

with open(path, 'w') as result_file:
    wr = csv.writer(result_file, dialect='excel')

    for q in bdata:
         wr.writerow(q)